In [1]:
## Self-Organizing Maps by Paras Chopra
## www.paraschopra.com
## paras1987@gmail.com
##
## Please give credit if you use my work.

from random import *
from math import *

class Node:

    def __init__(self, FV_size=10, PV_size=10, Y=0, X=0):
        self.FV_size=FV_size
        self.PV_size=PV_size
        self.FV = [0.0]*FV_size # Feature Vector
        self.PV = [0.0]*PV_size # Prediction Vector
        self.X=X # X location
        self.Y=Y # Y location
        
        for i in range(FV_size):
            self.FV[i]=random() # Assign a random number from 0 to 1
            
        for i in range(PV_size):
            self.PV[i]=random() # Assign a random number from 0 to 1


class SOM:

    #Let radius=False if you want to autocalculate the radis
    def __init__(self, height=10, width=10, FV_size=10, PV_size=10, radius=False, learning_rate=0.005):
        self.height=height
        self.width=width
        self.radius=radius if radius else (height+width)/2
        self.total=height*width
        self.learning_rate=learning_rate
        self.nodes=[0]*(self.total)
        self.FV_size=FV_size
        self.PV_size=PV_size
        for i in range(self.height):
            for j in range(self.width):
                self.nodes[(i)*(self.width)+j]=Node(FV_size, PV_size,i,j)

    # Train_vector format: [ [FV[0], PV[0]],
    #                        [FV[1], PV[1]], so on..
    
    def train(self, iterations=1000, train_vector=[[[0.0],[0.0]]]):
        time_constant=iterations/log(self.radius)
        radius_decaying=0.0
        learning_rate_decaying=0.0
        influence=0.0
        stack=[] #Stack for storing best matching unit's index and updated FV and PV
        temp_FV=[0.0]*self.FV_size
        temp_PV=[0.0]*self.PV_size
        for i in range(1,iterations+1):
            #print "Iteration number:",i
            radius_decaying=self.radius*exp(-1.0*i/time_constant)
            learning_rate_decaying=self.learning_rate*exp(-1.0*i/time_constant)
            print (i, end=', ')
            if i%50==0:
              print("")
            
            for  j in range(len(train_vector)):
                input_FV=train_vector[j][0]
                input_PV=train_vector[j][1]
                best=self.best_match(input_FV)
                stack=[]
                for k in range(self.total):
                    dist=self.distance(self.nodes[best],self.nodes[k])
                    if dist < radius_decaying:
                        temp_FV=[0.0]*self.FV_size
                        temp_PV=[0.0]*self.PV_size
                        influence=exp((-1.0*(dist**2))/(2*radius_decaying*i))

                        for l in range(self.FV_size):
                            #Learning
                            temp_FV[l]=self.nodes[k].FV[l]+influence*learning_rate_decaying*(input_FV[l]-self.nodes[k].FV[l])

                        for l in range(self.PV_size):
                            #Learning
                            temp_PV[l]=self.nodes[k].PV[l]+influence*learning_rate_decaying*(input_PV[l]-self.nodes[k].PV[l])

                        #Push the unit onto stack to update in next interval
                        stack[0:0]=[[[k],temp_FV,temp_PV]]

                
                for l in range(len(stack)):
                    
                    self.nodes[stack[l][0][0]].FV[:]=stack[l][1][:]
                    self.nodes[stack[l][0][0]].PV[:]=stack[l][2][:]

                
                                    

                

    #Returns prediction vector
    def predict(self, FV=[0.0],get_ij=False):
        best=self.best_match(FV)
        if get_ij:
          return self.nodes[best].PV,self.nodes[best].X,self.nodes[best].Y
        return self.nodes[best].PV
        
    #Returns best matching unit's index
    def best_match(self, target_FV=[0.0]):

        minimum=sqrt(self.FV_size) #Minimum distance
        minimum_index=1 #Minimum distance unit
        temp=0.0
        for i in range(self.total):
            temp=0.0
            temp=self.FV_distance(self.nodes[i].FV,target_FV)
            if temp<minimum:
                minimum=temp
                minimum_index=i

        
        return minimum_index

    def FV_distance(self, FV_1=[0.0], FV_2=[0.0]):
        temp=0.0
        for j in range(self.FV_size):
                temp=temp+(FV_1[j]-FV_2[j])**2

        temp=sqrt(temp)
        return temp

    def distance(self, node1, node2):
        return sqrt((node1.X-node2.X)**2+(node1.Y-node2.Y)**2)


In [2]:
print( "Initialization...")
a=SOM(5,5,2,1,False,0.05)

print( "Training for the XOR function..." )
a.train(100,[[[1,0],[1]],[[1,1],[0]],[[0,1],[1]],[[0,0],[0]]])

print( "\nPredictions for the XOR function..." )

print( "Prediction 0 0,", round(a.predict([0,0])[0]) )
print( "Prediction 1 0,", round(a.predict([1,0])[0]) )
print( "Prediction 0 1,", round(a.predict([0,1])[0]) )
print( "Prediction 1 1,", round(a.predict([1,1])[0]) )

Initialization...
Training for the XOR function...
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 

Predictions for the XOR function...
Prediction 0 0, 0
Prediction 1 0, 1
Prediction 0 1, 1
Prediction 1 1, 0


In [3]:
#Archivo numbers.py
#Script para obtener una lista de ceros y unos a partir de una regiÃ³n de 16x16 de una imagen
#Jorge Luis Rosas Trigueros
#23 marzo 2010


##from PIL import Image

##im=Image.open('usps_0.jpg')


##top=0     #Max=33
##left=0    #Max=32
##D=16
##
##muestra=[]
##
##for y in range(D):
####    row=[]
##    for x in range(D):        
##        p=im.getpixel((x+left*16,y+top*16))
##        if p > 128:
##            v=1
##        else:
##            v=0
###        row.append(v)
##        muestra.append(v)
##
##
##print muestra


##def get_sample(im,top,left,D):
##    muestra=[]
##
##    for y in range(0,D,2):
##    ##    row=[]
##        for x in range(0,D,2):        
##            p=im.getpixel((x+left*16,y+top*16))
##            if p > 128:
##                v=1
##            else:
##                v=0
##    #        row.append(v)
##            muestra.append(v)
##            
##    return muestra

    
conjunto_entrenamiento=[]

##im=Image.open('usps_0.jpg')
f= open('./usps_digits025/usps_0entrenamiento.txt','r')

##for x in range(5):
##    for y in range(4):
##        conjunto_entrenamiento.append([get_sample(im,y,x,16),[0]])
##        f.write(str(get_sample(im,y,x,16))[1:-1]+'\n')

for line in f:
##    print line
    muestra=[]
    for x in line.split(','):
        muestra.append(int(x))
    conjunto_entrenamiento.append([muestra,[0]])

f.close()
##im=Image.open('usps_2.jpg')

f= open('./usps_digits025/usps_2entrenamiento.txt','r')

##for x in range(5):
##    for y in range(4):
##        conjunto_entrenamiento.append([get_sample(im,y,x,16),[1]])
##        f.write(str(get_sample(im,y,x,16))[1:-1]+'\n')


for line in f:
    muestra=[]
    for x in line.split(','):
        muestra.append(int(x))
    conjunto_entrenamiento.append([muestra,[1]])

f.close()

##im=Image.open('usps_5.jpg')
f= open('./usps_digits025/usps_5entrenamiento.txt','r')

##for x in range(5):
##    for y in range(4):
##        conjunto_entrenamiento.append([get_sample(im,y,x,16),[2]])
##        f.write(str(get_sample(im,y,x,16))[1:-1]+'\n')

for line in f:
    muestra=[]
    for x in line.split(','):
        muestra.append(int(x))
    conjunto_entrenamiento.append([muestra,[2]])


f.close()


#from som import *

##a=SOM(5,5,2,1,False,0.05)
##
##print "Training for the XOR function..."
##a.train(100,[[[1,0],[1]],[[1,1],[0]],[[0,1],[1]],[[0,0],[0]]])
##
##print "Predictions for the XOR function..."
##
##print "Prediction 0 0,", round(a.predict([0,0])[0])
##print "Prediction 1 0,", round(a.predict([1,0])[0])
##print "Prediction 0 1,", round(a.predict([0,1])[0])
##print "Prediction 1 1,", round(a.predict([1,1])[0])

mymap=SOM( 15,12,64,1,False,0.05)

##im=Image.open('usps_0.jpg')
##muestra_nueva=get_sample(im,17,0,16)
f= open('./usps_digits025/usps_0prueba.txt','r')
muestra0=[]
for x in f.readline().split(','):
    muestra0.append(int(x))
f.close()
print( 'Predicción 1: ', mymap.predict(muestra0) )
##im=Image.open('usps_2.jpg')
##muestra_nueva=get_sample(im,17,0,16)
f= open('./usps_digits025/usps_2prueba.txt','r')
muestra1=[]
for x in f.readline().split(','):
    muestra1.append(int(x))
f.close()
print( 'Predicción 2: ', mymap.predict(muestra1) )
##im=Image.open('usps_5.jpg')
##muestra_nueva=get_sample(im,17,0,16)
f= open('./usps_digits025/usps_5prueba.txt','r')
muestra2=[]
for x in f.readline().split(','):
    muestra2.append(int(x))
f.close()
print( 'Predicción 3: ', mymap.predict(muestra2) )


mymap.train(500,conjunto_entrenamiento)

##im=Image.open('usps_0.jpg')
##muestra_nueva=get_sample(im,17,0,16)                                       
##f= open('usps_0prueba.txt','w')
##f.write(str(get_sample(im,17,0,16))[1:-1]+'\n')
##f.close()

print( '\nPredicción 1: ', mymap.predict(muestra0,True) )
##im=Image.open('usps_2.jpg')
##muestra_nueva=get_sample(im,17,0,16)
##f= open('usps_2prueba.txt','w')
##f.write(str(get_sample(im,17,0,16))[1:-1]+'\n')
##f.close()

print( 'Predicción 2: ', mymap.predict(muestra1,True) )
##im=Image.open('usps_5.jpg')
##muestra_nueva=get_sample(im,17,0,16)
##f= open('usps_5prueba.txt','w')
##f.write(str(get_sample(im,17,0,16))[1:-1]+'\n')
##f.close()

print( 'Predicción 3: ', mymap.predict(muestra2,True) )


map2D=[]

for y in range(12):
    row=[]
    for x in range(15):
        row.append(round(mymap.nodes[(x)*mymap.width+y].PV[0]))
    map2D.append(row)

for y in range(12):                                     
    print( map2D[y] )

Predicción 1:  [0.040841367592651556]
Predicción 2:  [0.12925393175129773]
Predicción 3:  [0.46855667703643966]
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 
101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 
151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199

In [4]:
for r in range(8):
  print(muestra1[r*8:r*8+8])

[0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 1, 1, 1, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 1]
